# Assignment 3: CNN

#### Student Name: Ryan Richardson

In this assignment, you will create a CNN model to classifiy the images in the CIFAR-10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html). It consists of 60,000 32x32 color images containing one of 10 object classes, (6000 images per class).


In [43]:
import pickle
import tensorflow as t
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

## Load the data

In [44]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
# Load the training and testing sets
trainingData = []
trainingLabels = []
testingData = []
testingLabels = []
for i in range(0, 5):
    batch = unpickle(f'E:/Chapman/graduate-work/cs-770-deep-learning/hw3/data/data_batch_{i + 1}')
    trainingData.append(batch[b'data'])
    trainingLabels += batch[b'labels']

batch = unpickle('E:/Chapman/graduate-work/cs-770-deep-learning/hw3/data/test_batch')
testingData.append(batch[b'data'])
testingLabels += batch[b'labels']
# Convert the data and labels to numpy arrays

trainingData = np.concatenate(trainingData)
trainingData = trainingData / 255.0
trainingData = np.reshape(trainingData, (-1, 3, 32, 32)).transpose(0, 2, 3, 1)

testingData = np.concatenate(testingData)
testingData = testingData / 255.0
testingData = np.reshape(testingData, (-1, 3, 32, 32)).transpose(0, 2, 3, 1)

trainingLabels = np.array(trainingLabels)
testingLabels = np.array(testingLabels)

In [53]:
print(trainingData.shape, trainingLabels.shape)
print(testingData.shape, testingLabels.shape)

(50000, 32, 32, 3) (50000,)
(10000, 32, 32, 3) (10000,)


## Train CNN Model

In [54]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Define the Model
Create a deep learning model with 5 convolutional layers, also add max pool layers after each conv layer or after two conv layers. At the end, add Fully connected layers and finally a softmax layer:

### I had an extremely rough time trying to get the data frame and the ImageDataGenerators to work properly, so I ended up settling with the following

In [47]:
# CODE HERE
# Define your model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.ZeroPadding2D(padding=((1, 1), (1, 1))),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.ZeroPadding2D(padding=((1, 1), (1, 1))),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.ZeroPadding2D(padding=((1, 1), (1, 1))),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.ZeroPadding2D(padding=((1, 1), (1, 1))),

    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile Your Model

In [50]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
early_stop= EarlyStopping(monitor='val_loss', mode ='min', verbose = 1, patience = 3)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only = True, mode ='min', verbose = 1)

# Train Your Model

In [51]:
history = model.fit(trainingData, trainingLabels, epochs=10,
                    validation_data=(testingData, testingLabels))


Epoch 1/10
1563/1563 [==============================] - 164s 103ms/step - loss: 1.0724 - accuracy: 0.6164 - val_loss: 0.9530 - val_accuracy: 0.6653
Epoch 2/10
1563/1563 [==============================] - 169s 108ms/step - loss: 0.8733 - accuracy: 0.6924 - val_loss: 0.8663 - val_accuracy: 0.7035
Epoch 3/10
1563/1563 [==============================] - 173s 111ms/step - loss: 0.7292 - accuracy: 0.7445 - val_loss: 0.8346 - val_accuracy: 0.7178
Epoch 4/10
1563/1563 [==============================] - 179s 115ms/step - loss: 0.6348 - accuracy: 0.7803 - val_loss: 0.8150 - val_accuracy: 0.7326
Epoch 5/10
1563/1563 [==============================] - 180s 115ms/step - loss: 0.5383 - accuracy: 0.8141 - val_loss: 0.8398 - val_accuracy: 0.7224
Epoch 6/10
1563/1563 [==============================] - 155s 99ms/step - loss: 0.4672 - accuracy: 0.8384 - val_loss: 0.8419 - val_accuracy: 0.7320
Epoch 7/10
1563/1563 [==============================] - 119s 76ms/step - loss: 0.4020 - accuracy: 0.8609 - val_lo

# Evaluate Your Model

In [52]:
loss, accuracy = model.evaluate(testingData, testingLabels, verbose=2)
print(accuracy)

313/313 - 6s - loss: 1.0157 - accuracy: 0.7355 - 6s/epoch - 19ms/step
0.7354999780654907
